### Markowitz Portfolio Optimization
#### Monte-Carlo Approach

In [1]:
# calculations
import pandas as pd
import numpy as np

# plotting 
import plotly
import plotly.graph_objects as go

# data
import yfinance as yf

from tqdm import tqdm

In [2]:
# import data
daily_returns = yf.download(["AAPL","JPM","WMT","TGT","MSFT","AMGN"], period= "5y", interval="1d")
dr = daily_returns["Adj Close"].pct_change()

[*********************100%***********************]  6 of 6 completed


In [3]:
# annulized retruns
mus = (1+dr.mean())**252 - 1 

# convariance
covar = dr.cov()*252
covar


,AAPL,AMGN,JPM,MSFT,TGT,WMT
AAPL,0.095976,0.036528,0.042082,0.065459,0.032147,0.024374
AMGN,0.036528,0.063712,0.031199,0.037407,0.025784,0.022267
JPM,0.042082,0.031199,0.094006,0.041512,0.030462,0.018230
MSFT,0.065459,0.037407,0.041512,0.080257,0.032273,0.024937
TGT,0.032147,0.025784,0.030462,0.032273,0.096222,0.030141
WMT,0.024374,0.022267,0.018230,0.024937,0.030141,0.048514


In [4]:
n_assets = 5
n_portfolios = 1000

mean_variance_pairs = []

np.random.seed(1993)

for i in range(n_portfolios):
    assets = np.random.choice(list(dr.columns), n_assets, replace = False)
    weights = np.random.rand(n_assets)
    weights = weights/sum(weights)

    portfolio_E_retruns = 0
    portfolio_E_variance = 0

    for i in range(len(assets)):
        portfolio_E_retruns += weights[i] * mus.loc[assets[i]]

        for j in range(len(assets)):
            portfolio_E_variance += weights[i] * weights[j] * covar.loc[assets[i],assets[j]]

    mean_variance_pairs.append([portfolio_E_retruns,portfolio_E_variance])

In [5]:
# mean_variance_pairs

In [6]:
# Plotting
mean_variance_pairs = np.array(mean_variance_pairs)

risk_free_rate=0 #-- Include risk free rate here

fig = go.Figure()
fig.add_trace(go.Scatter(x=mean_variance_pairs[:,1]**0.5, y=mean_variance_pairs[:,0], 
                      marker=dict(color=(mean_variance_pairs[:,0]-risk_free_rate)/(mean_variance_pairs[:,1]**0.5), 
                                  showscale=True, 
                                  size=7,
                                  line=dict(width=1),
                                  colorscale="RdBu",
                                  colorbar=dict(title="Sharpe<br>Ratio")
                                 ), 
                      mode='markers'))
fig.update_layout(template='plotly_white',
                  xaxis=dict(title='Annualised Risk (Volatility)'),
                  yaxis=dict(title='Annualised Return'),
                  title='Sample of Random Portfolios',
                  width=850,
                  height=500)
fig.update_xaxes(range=[0.18, 0.32])
fig.update_yaxes(range=[0.02,0.27])
fig.update_layout(coloraxis_colorbar=dict(title="Sharpe Ratio"))

In [7]:
n_assets = 5

mean_variance_pairs = []
weights_list=[]
tickers_list=[]

for i in tqdm(range(10000)):
    next_i = False
    while True:
        #- Choose assets randomly without replacement
        assets = np.random.choice(list(dr.columns), n_assets, replace=False)
        #- Choose weights randomly ensuring they sum to one
        weights = np.random.rand(n_assets)
        weights = weights/sum(weights)

        #-- Loop over asset pairs and compute portfolio return and variance
        portfolio_E_Variance = 0
        portfolio_E_Return = 0
        for i in range(len(assets)):
            portfolio_E_Return += weights[i] * mus.loc[assets[i]]
            for j in range(len(assets)):
                portfolio_E_Variance += weights[i] * weights[j] * covar.loc[assets[i], assets[j]]

        #-- Skip over dominated portfolios
        for R,V in mean_variance_pairs:
            if (R > portfolio_E_Return) & (V < portfolio_E_Variance):
                next_i = True
                break
        if next_i:
            break

        #-- Add the mean/variance pairs to a list for plotting
        mean_variance_pairs.append([portfolio_E_Return, portfolio_E_Variance])
        weights_list.append(weights)
        tickers_list.append(assets)
        break

100%|██████████| 10000/10000 [00:02<00:00, 3561.80it/s]


In [8]:
mean_variance_pairs = np.array(mean_variance_pairs)

risk_free_rate=0 #-- Include risk free rate here

fig = go.Figure()
fig.add_trace(go.Scatter(x=mean_variance_pairs[:,1]**0.5, y=mean_variance_pairs[:,0], 
                      marker=dict(color=(mean_variance_pairs[:,0]-risk_free_rate)/(mean_variance_pairs[:,1]**0.5), 
                                  showscale=True, 
                                  size=7,
                                  line=dict(width=1),
                                  colorscale="RdBu",
                                  colorbar=dict(title="Sharpe<br>Ratio")
                                 ), 
                      mode='markers',
                      text=[str(np.array(tickers_list[i])) + "<br>" + str(np.array(weights_list[i]).round(2)) for i in range(len(tickers_list))]))
fig.update_layout(template='plotly_white',
                  xaxis=dict(title='Annualised Risk (Volatility)'),
                  yaxis=dict(title='Annualised Return'),
                  title='Sample of Random Portfolios',
                  width=850,
                  height=500)
fig.update_xaxes(range=[0.18, 0.35])
fig.update_yaxes(range=[0.05,0.29])
fig.update_layout(coloraxis_colorbar=dict(title="Sharpe Ratio"))

In [9]:
oko = yf.download("0P0001MOZR.L", period= "1y", interval= "1d")

[*********************100%***********************]  1 of 1 completed


In [10]:
oko

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-03-15,99.194397,99.194397,99.194397,99.194397,99.194397,0
2022-03-16,101.783897,101.783897,101.783897,101.783897,101.783897,0
2022-03-18,102.964897,102.964897,102.964897,102.964897,102.964897,0
2022-03-21,10357.169922,10357.169922,10357.169922,10357.169922,10357.169922,0
2022-03-22,10357.400391,10357.400391,10357.400391,10357.400391,10357.400391,0
2022-03-23,10315.889648,10315.889648,10315.889648,10315.889648,10315.889648,0
2022-03-25,10379.870117,10379.870117,10379.870117,10379.870117,10379.870117,0
2022-03-28,10439.870117,10439.870117,10439.870117,10439.870117,10439.870117,0
2022-03-29,10634.669922,10634.669922,10634.669922,10634.669922,10634.669922,0


#### Temperature Targeting Program
$$ 
max \left( x' \mu - \frac{RA}{2} x' \sum x \right)  \\
s.t. \; x'T \leq \bar{T} \\
\; \; x \geq 0 \\
\\
x' \mu \;: \; \text{Expected Return} \\
\frac{RA}{2} x' \sum x \; : \; \text{Risk aversion} \\
\bar{T} \; : \; 

$$

#### TA Utility 
$$
max \left( x' \mu - \frac{RA}{2} x' \sum x - \; TA \; x'T \right)  \\
s.t. \; x \geq 0
$$

In [25]:
from cvxopt import matrix, solvers

In [17]:
# Create Matrix
A = matrix(range(16),(4,4))

<4x4 matrix, tc='i'>

In [21]:
# numpy array from matrix
B = np.array(A)
type(B)

numpy.ndarray

In [23]:
# matrix from numpy array
C = matrix(B)
type(C)

cvxopt.base.matrix

In [29]:
# solving linear problem
A = matrix([ [-1.0, -1.0, 0.0, 1.0], [1.0, -1.0, -1.0, -2.0] ])
b = matrix([ 1.0, -2.0, 0.0, 4.0 ])
c = matrix([ 2.0, 1.0 ])

sol=solvers.lp(c,A,b)

print(sol["x"])

     pcost       dcost       gap    pres   dres   k/t
 0:  2.6471e+00 -7.0588e-01  2e+01  8e-01  2e+00  1e+00
 1:  3.0726e+00  2.8437e+00  1e+00  1e-01  2e-01  3e-01
 2:  2.4891e+00  2.4808e+00  1e-01  1e-02  2e-02  5e-02
 3:  2.4999e+00  2.4998e+00  1e-03  1e-04  2e-04  5e-04
 4:  2.5000e+00  2.5000e+00  1e-05  1e-06  2e-06  5e-06
 5:  2.5000e+00  2.5000e+00  1e-07  1e-08  2e-08  5e-08
Optimal solution found.
[ 5.00e-01]
[ 1.50e+00]



In [30]:
# solving quadratic problem

Q = 2*matrix([ [2, .5], [.5, 1] ])
p = matrix([1.0, 1.0])
G = matrix([[-1.0,0.0],[0.0,-1.0]])
h = matrix([0.0,0.0])
A = matrix([1.0, 1.0], (1,2))
b = matrix(1.0)

sol=solvers.qp(Q, p, G, h, A, b)

print(sol["x"])

     pcost       dcost       gap    pres   dres
 0:  1.8889e+00  7.7778e-01  1e+00  2e-16  2e+00
 1:  1.8769e+00  1.8320e+00  4e-02  2e-16  6e-02
 2:  1.8750e+00  1.8739e+00  1e-03  2e-16  5e-04
 3:  1.8750e+00  1.8750e+00  1e-05  1e-16  5e-06
 4:  1.8750e+00  1.8750e+00  1e-07  1e-16  5e-08
Optimal solution found.
[ 2.50e-01]
[ 7.50e-01]

